In [14]:
import pandas as pd
master_data = pd.read_excel("../0_dataset_master/Experiment_edited_V7_rr_with_correction.xlsx")
slice_data = master_data[['End Date',"Experiment set",'Experiment id', "Time_thermal"]]
slice_data.head()

,End Date,Experiment set,Experiment id,Time_thermal
0,2023-07-10 11:02:50,set1,id2,10:45-11:05
1,2023-07-10 11:03:41,set1,id6,10:45-11:05
2,2023-07-10 11:03:52,set1,id1,10:45-11:05
3,2023-07-10 11:04:08,set1,id5,10:45-11:05
4,2023-07-10 11:05:05,set1,id3,10:45-11:05


In [15]:
data_set = {}
ID_arr = [1,2,3,4,5,6]
new_set = [1,2,3,4,5,6]
for set_ in new_set:
    data_set["set"+str(set_)]={}
    for id_ in ID_arr:
        data_set["set"+str(set_)]["id"+str(id_)] = pd.read_excel(f"../7_resampleIntoEachSet/SkinT_set{set_}.xlsx", sheet_name=f"id{id_}", parse_dates=['Time'], date_format="mixed")

In [16]:
from datetime import datetime
refineset = ["set1","set2"]
for set_ in refineset:
    for id_ in ID_arr:
        date_result = []
        # print(f"process on {set_} id{id_}")
        for _, row in data_set[set_]["id"+str(id_)].iterrows():
            date_ = row['Time']
            if date_.day == 13:
                date_result.append(date_)
                continue    
            date_ = datetime.strptime(str(date_), "%Y-%d-%m %H:%M:%S")
            date_result.append(date_)
        data_set[set_]["id"+str(id_)].Time=date_result

In [20]:
from datetime import datetime
result = []
try:
    for _, row in slice_data.iterrows():
        date = str(row["End Date"].date())
        start_clock, end_clock = row['Time_thermal'].split("-")
        start_time = date+" "+start_clock
        end_time = date+" "+end_clock

        end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")

        # print(row)
        # print(start_time)
        # print(end_time)

        dataFiltered = data_set[row['Experiment set']][row['Experiment id']][(end_time > data_set[row['Experiment set']][row['Experiment id']]['Time']) & (start_time <= data_set[row['Experiment set']][row['Experiment id']]['Time'])]
        # print(dataFiltered)
        # break
        resultTemp = []
        try:
            resultTemp.append(dataFiltered[1].mean(axis=0))
        except KeyError:
            resultTemp.append(dataFiltered['1'].mean(axis=0))
        
        try:
            resultTemp.append(dataFiltered[2].mean(axis=0))
        except KeyError:
            resultTemp.append(dataFiltered['2'].mean(axis=0))
        
        try:
            resultTemp.append(dataFiltered[3].mean(axis=0))
        except KeyError:
            resultTemp.append(dataFiltered['3'].mean(axis=0))
        
        try:
            resultTemp.append(dataFiltered[4].mean(axis=0))
        except KeyError:
            resultTemp.append(dataFiltered['4'].mean(axis=0))
        
        result.append(resultTemp)
except ValueError:
    print(row) 

master_data[['Average_1','Average_2','Average_3','Average_4']] = result

In [21]:
master_data.to_excel("../0_dataset_master/Experiment_edited_V8.xlsx", index=False)